### Elementy sztucznej inteligencji
# Projekt: Binarne Drzewa Decyzyjne
autorzy:
1. Arkadiusz Florek
2. Maciej Komosa
3. Albert Pieniądz
4. Jakub Zięba

## TODO: Id3, opis jak działa wezel
### błędy:
1. dane nie są formatowane jak dane w przykładzie w linku (na dole)
2. prawdopodobniej ostatecznie trzeba będzie usunąć klasę, bo trzeba było to zrobic w skrypcie, ale w klasie lepiej widać co jest grane
### możliwe rozwiązania: zastosować formatowanie z maila od prof'a albo jakieś prostsze

In [149]:
import numpy as np, pandas as pd

Klasa Węzła

In [162]:
class Wezel:
    def __init__(self):
        self.wartosc = None
        self.kolejny = None
        self.dzieci = []

Obliczanie entropii

In [151]:
from math import log2

def _entropia(dane):
        # Tworzymy słownik, w którym kluczem jest wartość, a wartością liczba wystąpień tej wartości
        slownikWartosci = {}
        for _ in dane:
            atrybut = _[-1]
            if atrybut not in slownikWartosci:
                slownikWartosci[atrybut] = 0
            slownikWartosci[atrybut] += 1
    
        # Obliczamy entropię dla danej tabeli ze wzoru
        entropia = 0.0
        for _ in slownikWartosci.values():
            p = _ / len(dane)
            entropia -= p * log2(p)
    
        return entropia

Obliczanie zysku informacji

In [152]:
def _zysk_informacji(dane, przeslanka):
        # Tworzymy słownik, w którym kluczem jest wartość, a wartością lista wierszy z tą wartością
        wartosci_przeslanek = {}
        for wiersz in [*range(len(dane))]:
            wartosc = dane[wiersz, przeslanka]
            if wartosc not in wartosci_przeslanek:
                wartosci_przeslanek[wartosc] = []
            wartosci_przeslanek[wartosc].append(dane[wiersz])
    
        # Obliczamy entropię dla każdego podziału i sumujemy, aby uzyskać entropię po podziale
        ilosc_wierszy = len(dane)
        entropie = []
        for wartosci_przeslanki in wartosci_przeslanek.values():
            ilosc_wartosci_przeslanki = len(wartosci_przeslanki)
            entropia_przeslanki = _entropia(wartosci_przeslanki)
            entropie.append((ilosc_wartosci_przeslanki / ilosc_wierszy) * entropia_przeslanki)
        entropia_przeslanki = sum(entropie)
    
        # Obliczamy przyrost informacji
        zysk = _entropia(dane) - entropia_przeslanki
    
        return zysk

Obliczanie przeslanki o najwiekszym zysku

In [153]:
def _przeslanka_o_najwiekszym_zysku(dane, nazwy_przeslanek, przeslanki_indeksy):
        
        # tabela przeslanek, czyli tabela o wartosciach
        # z zakresu dlugosci jednego wiersza odjąć konkluzję

        entropie_przeslanek = [_zysk_informacji(dane, przeslanka) for przeslanka in przeslanki_indeksy]
        indeks_przeslanka_o_najwiekszym_zysku = przeslanki_indeksy[entropie_przeslanek.index(max(entropie_przeslanek))]

        return nazwy_przeslanek[indeks_przeslanka_o_najwiekszym_zysku], indeks_przeslanka_o_najwiekszym_zysku

Algorytm id3 i jego rekursywny modul

In [184]:
def _id3(dane, nazwy_przeslanek, konkluzje, wezel):
    
        # indeksy
        dane_indeksy =[*range(len(dane))]

        #przeslanki
        przeslanki_indeksy = [*range(len(nazwy_przeslanek))]
        return _id3_rekursja(dane, dane_indeksy,
                                   przeslanki_indeksy,
                                   wezel)

def _id3_rekursja(dane, dane_indeksy, przeslanki_indeksy, wezel):
        if not wezel:
            wezel = Wezel()  # inicjalizacja wezla
        # konkluzje
        klasy_w_przeslankach = dane[:, -1]
        # jesli wszystkie wiersze maja ta sama konkluzje
        if len(set(klasy_w_przeslankach)) == 1:
            wezel.wartosc = dane[dane_indeksy[0], -1]
            return wezel
        if len(przeslanki_indeksy) == 0:
        # jesli nie ma juz wiecej do policzenia
            wezel.wartosc = max(set(klasy_w_przeslankach), key=list(klasy_w_przeslankach).count)
            return wezel
        # wybieramy przeslankę o największym zysku
        najlepsza_przeslanka, indeks_najlepszej_przeslanki = _przeslanka_o_najwiekszym_zysku(dane, dane_indeksy, przeslanki_indeksy)
        wezel.wartosc = najlepsza_przeslanka
        wezel.dzieci = []
        # wartosc przeslanki dla kazdego wiersza
        wartosci_przeslanek = list(set([dane[i][indeks_najlepszej_przeslanki] for i in dane_indeksy]))
        # pętla przez wszystkie wartosci przeslanki
        for wartosc in wartosci_przeslanek:
            dziecko = Wezel()
            dziecko.wartosc = wartosc
            wezel.dzieci.append(dziecko)
            dziecko_dane = np.array([i for i in dane_indeksy if dane[i][indeks_najlepszej_przeslanki] == wartosc])
            if not dziecko_dane.any():
                dziecko.kolejny = max(set([klasy_w_przeslankach]), key=list(klasy_w_przeslankach).count)
            else:
                if przeslanki_indeksy and indeks_najlepszej_przeslanki in przeslanki_indeksy:
                    do_wyciecia = przeslanki_indeksy.index(indeks_najlepszej_przeslanki)
                    przeslanki_indeksy.pop(do_wyciecia)
                # rekursja
                dziecko.kolejny = _id3_rekursja(dane, dziecko_dane, przeslanki_indeksy, dziecko.kolejny)
        return wezel

Graf

In [174]:
from graphviz import Digraph

def _dot_graph(node, dot=None):
    if dot is None:
        dot = Digraph(comment='ID3 Decision Tree')
    if node is None:
        return dot
    else:
        dot.node(str(id(node)), str(node.wartosc))
        for child_node in node.dzieci:
            _dot_graph(child_node.kolejny, dot)
            dot.edge(str(id(node)), str(id(child_node)), label=str(node.wartosc))
    return dot

Zadanie

In [185]:
#pobieranie danych z nagłówkami i przesyłanie ich do funkcji - dane i nagłówki oddzielnie - trochę śmie
dane = pd.read_csv("sport.csv", header=None).to_numpy()
drzewo = Wezel()
print("entropia: ", _entropia(dane[1:, -1])) # <- działa
print("zysk informacji:")
for i in range(len(dane[1, :-1])):
    print(i, _zysk_informacji(dane[1:, :], i)) # <- działa
print("przeslanka p najwiekszym zysku:", _przeslanka_o_najwiekszym_zysku(dane[1:, :], dane[0, :-1], [*range(len(dane[0, :-1]))])) # <- działa
drzewo = _id3(dane[1:, :], dane[0, :-1], dane[1:, -1], drzewo) # <- działa??
for _ in drzewo.dzieci:
    print(_.kolejny.wartosc)

entropia:  2.67943596719348
zysk informacji:
0 0.1651500355641029
1 0.06226147757579792
2 0.09413277111930896
3 0.06929896561476978
4 0.21714308177218555
przeslanka p najwiekszym zysku: ('typ treningu', 4)
dziecko silowy
dziecko miasto
dziecko 3-5
dziecko w parze
dziecko zaawansowany
3 if: 2
dziecko amator
3 if: 2
dziecko poczatkujacy
3 if: 2
dziecko indywidualnie
3 if: 2
dziecko w grupie
3 if: 2
dziecko <2
3 if: 2
dziecko >5
3 if: 2
dziecko wies
3 if: 2
dziecko rekreacyjny
3 if: 2
dziecko wytrzymalosciowy
3 if: 2
dziecko techniczny
3 if: 2
108
2
2
2
